In [58]:
import numpy as np
import sys
import yaml
import random

In [82]:
input_file = 'inputs/expe/input_100ag_1dy_5sta_32_seed1.yaml'
dy_file = 'dy_obstacles/expe/dy_100ag_1dy_5sta_32_seed1.yaml'

In [83]:
N = 100
static_obs_ratio = 0.05
map_size_x = 32
map_size_y = 32
x_min = 0
y_min = 0
seed = 1


num_dynamic_obs = 1
step_num_from = 0
step_num_to = 1
manhadon_dist_min = 3

In [59]:
def write_all(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min):
    input_f = open(input_file, 'a')

    input_dict = {'agents': []}

    start_ls = []
    end_ls = []

    input_f.write('agents:\n')
#     print('agents:')
    for agent_num in range(N):
        while True:
            start_x = random.choice(range(x_min, map_size_x))
            start_y = random.choice(range(y_min, map_size_y))
            end_x = random.choice(range(x_min, map_size_x))
            end_y = random.choice(range(y_min, map_size_y))
            if [start_x, start_y] not in start_ls and [end_x, end_y] not in end_ls:
                start_ls.append([start_x, start_y])
                end_ls.append([end_x, end_y])
                break
            else:
                continue
        input_dict['agents'].append({'start': [start_x, start_y], 'goal': [end_x, end_y], 'name': "agent" + str(agent_num)})
        input_f.write("-   start: " + str([start_x, start_y]) + '\n')
        input_f.write("    goal: " + str([end_x, end_y]) + '\n')
        input_f.write("    name: agent" + str(agent_num) + '\n')
#         print("-   start: ", [start_x, start_y])
#         print("    goal: ", [end_x, end_y])
#         print("    name: agent" + str(agent_num))
        
        
        
        
        
        
        
    num_static_obs = int(static_obs_ratio * (map_size_x * map_size_y))
    # num_static_obs = 3
    obs_ls = []
    input_f.write('map:\n')
    input_f.write('    dimensions: [' + str(map_size_x) + ', ' + str(map_size_y) + ']\n')
    input_f.write('    obstacles:' + '\n')
#     print('map:')
#     print('    dimensions: [', map_size_x, ', ', map_size_y, ']')
#     print('    obstacles:')
    for i in range(num_static_obs):
        while True:
            obs_x = random.choice(range(map_size_x))
            obs_y = random.choice(range(map_size_y))
            if [obs_x, obs_y] not in start_ls and [obs_x, obs_y] not in end_ls and [obs_x, obs_y] not in obs_ls:
                obs_ls.append([obs_x, obs_y])
                break
            else:
                continue
        input_f.write('    - !!python/tuple ' + str([obs_x, obs_y]) + '\n')
#         print('    - !!python/tuple ', [obs_x, obs_y])


        
        
        
        
    input_f.write('obstacles:\n')
    for i in range(num_dynamic_obs):
        input_f.write('-   start: [0, 0]\n')
        input_f.write('    goal: [1000, 1000]\n')
        input_f.write('    name: obstacle' + str(i) + '\n')

    input_f.close()

    dy_f = open(dy_file, 'a')

#     random.seed(seed)

    dy_f.write('schedule:\n')
#     print('schedule:')
    for dyn_obs in range(num_dynamic_obs):
        dy_f.write('  obstacle' + str(dyn_obs) + ':\n')
#         print('  obstacle' + str(dyn_obs) + ':')
        while True:
            step_x = random.choice(range(map_size_x))
            step_y = random.choice(range(map_size_y))
            step = [step_x, step_y]

            is_bigger = True
            for start_pt in start_ls:
                manhadon_dist = abs(start_pt[0] - step_x) + abs(start_pt[1] - step_y)
                if manhadon_dist < manhadon_dist_min:
                    is_bigger = False
                    break

            if is_bigger:
                if step not in start_ls and step not in end_ls and step not in obs_ls:
                    dy_f.write('  - t: ' + str(0) + '\n')
                    dy_f.write('    x: ' + str(step_x) + '\n')
                    dy_f.write('    y: ' + str(step_y) + '\n')
#                     print('  - t: ', 0)
#                     print('    x: ', step_x)
#                     print('    y: ', step_y)
                    break
        step_num = random.choice(range(step_num_from, step_num_to))
        for step_ in range(step_num):
            while True:
                direction = random.choice(range(5)) # 0 => up | 1 => down | 2 => left | 3 => right | 4 => still 
                if direction == 0:
                    step_x_ = step_x
                    step_y_ = step_y + 1 
                elif direction == 1:
                    step_x_ = step_x
                    step_y_ = step_y - 1 
                elif direction == 2:
                    step_y_ = step_y
                    step_x_ = step_x - 1 
                elif direction == 3:
                    step_y_ = step_y
                    step_x_ = step_x + 1 
                else:
                    step_y_ = step_y
                    step_x_ = step_x

                step = [step_x_, step_y_]

                if step not in end_ls and step not in obs_ls and (0 <= step_x_ < map_size_x) and (0 <= step_y_ < map_size_y):
                    dy_f.write('  - t: ' + str((step_ + 1)) + '\n')
                    dy_f.write('    x: ' + str(step_x) + '\n')
                    dy_f.write('    y: ' + str(step_y) + '\n')
#                     print('  - t: ', (step_ + 1))
#                     print('    x: ', step_x_)
#                     print('    y: ', step_y_)
                    step_x = step_x_
                    step_y = step_y_
                    break
    dy_f.close()
    
def write_dy_solo(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min):

    dy_f = open(dy_file, 'a')

#     random.seed(seed)

    dy_f.write('schedule:\n')
#     print('schedule:')
    for dyn_obs in range(num_dynamic_obs):
        dy_f.write('  obstacle' + str(dyn_obs) + ':\n')
        dy_f.write('  - t: 0\n')
        dy_f.write('    x: 1000\n')
        dy_f.write('    y: 1000\n')
#         print('  obstacle' + str(dyn_obs) + ':')
#         while True:
#             step_x = random.choice(range(map_size_x))
#             step_y = random.choice(range(map_size_y))
#             step = [step_x, step_y]

#             is_bigger = True
#             for start_pt in start_ls:
#                 manhadon_dist = abs(start_pt[0] - step_x) + abs(start_pt[1] - step_y)
#                 if manhadon_dist < manhadon_dist_min:
#                     is_bigger = False
#                     break

#             if is_bigger:
#                 if step not in start_ls and step not in end_ls and step not in obs_ls:
#                     dy_f.write('  - t: ' + str(0) + '\n')
#                     dy_f.write('    x: ' + str(step_x) + '\n')
#                     dy_f.write('    y: ' + str(step_y) + '\n')
# #                     print('  - t: ', 0)
# #                     print('    x: ', step_x)
# #                     print('    y: ', step_y)
#                     break
#         step_num = random.choice(range(step_num_from, step_num_to))
#         for step_ in range(step_num):
#             while True:
#                 direction = random.choice(range(5)) # 0 => up | 1 => down | 2 => left | 3 => right | 4 => still 
#                 if direction == 0:
#                     step_x_ = step_x
#                     step_y_ = step_y + 1 
#                 elif direction == 1:
#                     step_x_ = step_x
#                     step_y_ = step_y - 1 
#                 elif direction == 2:
#                     step_y_ = step_y
#                     step_x_ = step_x - 1 
#                 elif direction == 3:
#                     step_y_ = step_y
#                     step_x_ = step_x + 1 
#                 else:
#                     step_y_ = step_y
#                     step_x_ = step_x

#                 step = [step_x_, step_y_]

#                 if step not in end_ls and step not in obs_ls and (0 <= step_x_ < map_size_x) and (0 <= step_y_ < map_size_y):
#                     dy_f.write('  - t: ' + str((step_ + 1)) + '\n')
#                     dy_f.write('    x: ' + str(step_x) + '\n')
#                     dy_f.write('    y: ' + str(step_y) + '\n')
# #                     print('  - t: ', (step_ + 1))
# #                     print('    x: ', step_x_)
# #                     print('    y: ', step_y_)
#                     step_x = step_x_
#                     step_y = step_y_
#                     break
    dy_f.close()

In [55]:
import os
import random
N = 20
static_obs_ratio = 0.01
map_size_x = 32
map_size_y = 32
x_min = 0
y_min = 0
seed = 1

num_dynamic_obs = 10
step_num_from = 0
step_num_to = 1
manhadon_dist_min = 3

tag_initial = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
    + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x)
    
os.mkdir('inputs/table/' + tag_initial)
os.mkdir('outputs/table/' + tag_initial)
os.mkdir('dy_obstacles/table/' + tag_initial)

shell_root = './shells/'
sh_f = open(shell_root + tag_initial + '.sh', 'a')
sh_f.write('cd ../ \n')
for seed in range(20):
    random.seed(seed)

    tag = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
        + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x) + '_seed' + str(seed)
    
    input_file = 'inputs/table/' + tag_initial + '/input_' + tag + '.yaml'
    dy_file = 'dy_obstacles/table/' + tag_initial + '/dy_' + tag + '.yaml'
    
    write_all(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min)
    
    sh_f.write('python3 multi_thread_mapf.py inputs/table/' + tag_initial + '/input_' + tag + '.yaml ' + \
              'outputs/table/' + tag_initial + '/output_' + tag + '.yaml ' + \
              'dy_obstacles/table/' + tag_initial + '/dy_' + tag + '.yaml ' + \
              'results/results_' + tag_initial + '.txt \n')
    
sh_f.close()
    

In [64]:
# DBCBS
import os
import random
N = 20
static_obs_ratio = 0.05
map_size_x = 64
map_size_y = 64
x_min = 0
y_min = 0
seed = 1

num_dynamic_obs = 50
step_num_from = 20
step_num_to = 30
manhadon_dist_min = 3


for num_dynamic_obs in [10, 20, 30, 40]:
    tag_initial = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
        + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x) + '_manha' + str(manhadon_dist_min)

    if os.path.exists('inputs/final/' + tag_initial):
        print('inputs/final/' + tag_initial + ' EXIST~')
    else:    
        os.mkdir('inputs/final/' + tag_initial)
        
    if os.path.exists('outputs/final/' + tag_initial):
        print('outputs/final/' + tag_initial + ' EXIST~')
    else:
        os.mkdir('outputs/final/' + tag_initial)
        
    if os.path.exists('dy_obstacles/final/' + tag_initial):
        print('dy_obstacles/final/' + tag_initial + ' EXIST~')
    else:
        os.mkdir('dy_obstacles/final/' + tag_initial)

    shell_root = './shells/final/'
    sh_f = open(shell_root + tag_initial + '.sh', 'a')
    sh_f.write('cd ../../ \n')
    for seed in range(20):
        random.seed(seed)

        tag = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
            + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x) + '_seed' + str(seed) + '_manha' + str(manhadon_dist_min)

        input_file = 'inputs/final/' + tag_initial + '/input_' + tag + '.yaml'
        dy_file = 'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '.yaml'
#         dy_file_without = 'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '_without.yaml'

        write_all(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min)
#         write_dy_solo(input_file, dy_file_without, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, 1, 0, 1, 10)

        sh_f.write('python3 multi_thread_mapf_dy.py inputs/final/' + tag_initial + '/input_' + tag + '.yaml ' + \
                  'outputs/final/' + tag_initial + '/output_' + tag + '.yaml ' + \
                  'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '.yaml \n')

#         sh_f.write('python3 multi_thread_mapf_dy.py inputs/final/' + tag_initial + '/input_' + tag + '.yaml ' + \
#                   'outputs/final/' + tag_initial + '/output_' + tag + '_without.yaml ' + \
#                   'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '_without.yaml \n')

#         sh_f.write('python3 calc_collision.py ' + 'outputs/final/' + tag_initial + '/output_' + tag + '.yaml ' + \
#                   'outputs/final/' + tag_initial + '/output_' + tag + '_without.yaml ' + \
#                   'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '.yaml ' + \
#                   'results/final/results_' + tag_initial + '_dy.txt \n')


        sh_f.write('python3 calc_collision.py ' + 'outputs/final/' + tag_initial + '/output_' + tag + '.yaml ' + \
                  'outputs/table/20ag_50dy_5sta_32/output_20ag_50dy_5sta_32_seed0_without.yaml ' + \
                  'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '.yaml ' + \
                  'results/final/results_' + tag_initial + '_dy.txt \n')
    sh_f.close()



In [69]:
# ABCBS
import os
import random
N = 20
static_obs_ratio = 0.05
map_size_x = 64
map_size_y = 64
x_min = 0
y_min = 0
seed = 1

num_dynamic_obs = 50
step_num_from = 20
step_num_to = 30
manhadon_dist_min = 3


for num_dynamic_obs in [10, 20, 30, 40]:
    tag_initial = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
        + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x) + '_manha' + str(manhadon_dist_min)

    if os.path.exists('inputs/final_abcbs/' + tag_initial):
        print('inputs/final_abcbs/' + tag_initial + ' EXIST~')
    else:    
        os.mkdir('inputs/final_abcbs/' + tag_initial)
        
    if os.path.exists('outputs/final_abcbs/' + tag_initial):
        print('outputs/final_abcbs/' + tag_initial + ' EXIST~')
    else:
        os.mkdir('outputs/final_abcbs/' + tag_initial)
        
    if os.path.exists('dy_obstacles/final_abcbs/' + tag_initial):
        print('dy_obstacles/final_abcbs/' + tag_initial + ' EXIST~')
    else:
        os.mkdir('dy_obstacles/final_abcbs/' + tag_initial)

    shell_root = './shells/final_abcbs/'
    sh_f = open(shell_root + tag_initial + '.sh', 'a')
    sh_f.write('cd ../../ \n')
    for seed in range(20):
        random.seed(seed)

        tag = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
            + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x) + '_seed' + str(seed) + '_manha' + str(manhadon_dist_min)

        input_file = 'inputs/final_abcbs/' + tag_initial + '/input_' + tag + '.yaml'
        dy_file = 'dy_obstacles/final_abcbs/' + tag_initial + '/dy_' + tag + '.yaml'
#         dy_file_without = 'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '_without.yaml'

        write_all(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min)
#         write_dy_solo(input_file, dy_file_without, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, 1, 0, 1, 10)

        sh_f.write('python3 cbs_mine_dy.py inputs/final_abcbs/' + tag_initial + '/input_' + tag + '.yaml ' + \
                  'outputs/final_abcbs/' + tag_initial + '/output_' + tag + '.yaml ' + \
                  'dy_obstacles/final_abcbs/' + tag_initial + '/dy_' + tag + '.yaml ' + str(seed) + " \n")

#         sh_f.write('python3 multi_thread_mapf_dy.py inputs/final/' + tag_initial + '/input_' + tag + '.yaml ' + \
#                   'outputs/final/' + tag_initial + '/output_' + tag + '_without.yaml ' + \
#                   'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '_without.yaml \n')

#         sh_f.write('python3 calc_collision.py ' + 'outputs/final/' + tag_initial + '/output_' + tag + '.yaml ' + \
#                   'outputs/final/' + tag_initial + '/output_' + tag + '_without.yaml ' + \
#                   'dy_obstacles/final/' + tag_initial + '/dy_' + tag + '.yaml ' + \
#                   'results/final/results_' + tag_initial + '_dy.txt \n')


        sh_f.write('python3 calc_collision.py ' + 'outputs/final_abcbs/' + tag_initial + '/output_' + tag + '.yaml ' + \
                  'outputs/table/20ag_50dy_5sta_32/output_20ag_50dy_5sta_32_seed0_without.yaml ' + \
                  'dy_obstacles/final_abcbs/' + tag_initial + '/dy_' + tag + '.yaml ' + \
                  'results/final_abcbs/results_' + tag_initial + '_dy.txt \n')
    sh_f.close()



In [47]:
a = [1, 2, 3]
a[:0]

[]

In [57]:
'2' * 2.12


TypeError: can't multiply sequence by non-int of type 'float'

In [84]:
random.seed(seed)
# def write_input(N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min):
input_f = open(input_file, 'a')

input_dict = {'agents': []}

start_ls = []
end_ls = []

input_f.write('agents:\n')
print('agents:')
for agent_num in range(N):
    while True:
        start_x = random.choice(range(x_min, map_size_x))
        start_y = random.choice(range(y_min, map_size_y))
        end_x = random.choice(range(x_min, map_size_x))
        end_y = random.choice(range(y_min, map_size_y))
        if [start_x, start_y] not in start_ls and [end_x, end_y] not in end_ls:
            start_ls.append([start_x, start_y])
            end_ls.append([end_x, end_y])
            break
        else:
            continue
    input_dict['agents'].append({'start': [start_x, start_y], 'goal': [end_x, end_y], 'name': "agent" + str(agent_num)})
    input_f.write("-   start: " + str([start_x, start_y]) + '\n')
    input_f.write("    goal: " + str([end_x, end_y]) + '\n')
    input_f.write("    name: agent" + str(agent_num) + '\n')
    print("-   start: ", [start_x, start_y])
    print("    goal: ", [end_x, end_y])
    print("    name: agent" + str(agent_num))


# print(start_ls)
# print(end_ls)

agents:
-   start:  [8, 4]
    goal:  [16, 7]
    name: agent0
-   start:  [31, 28]
    goal:  [30, 24]
    name: agent1
-   start:  [13, 6]
    goal:  [31, 1]
    name: agent2
-   start:  [24, 27]
    goal:  [0, 28]
    name: agent3
-   start:  [17, 14]
    goal:  [6, 20]
    name: agent4
-   start:  [1, 1]
    goal:  [1, 0]
    name: agent5
-   start:  [24, 13]
    goal:  [27, 1]
    name: agent6
-   start:  [14, 28]
    goal:  [31, 14]
    name: agent7
-   start:  [22, 14]
    goal:  [14, 29]
    name: agent8
-   start:  [18, 1]
    goal:  [26, 6]
    name: agent9
-   start:  [11, 18]
    goal:  [7, 21]
    name: agent10
-   start:  [27, 12]
    goal:  [19, 18]
    name: agent11
-   start:  [31, 25]
    goal:  [2, 30]
    name: agent12
-   start:  [15, 25]
    goal:  [26, 11]
    name: agent13
-   start:  [23, 23]
    goal:  [5, 28]
    name: agent14
-   start:  [6, 10]
    goal:  [25, 23]
    name: agent15
-   start:  [31, 1]
    goal:  [30, 2]
    name: agent16
-   start:  [19, 25

In [85]:
# def write_sta_obs(N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min):
num_static_obs = int(static_obs_ratio * (map_size_x * map_size_y))
# num_static_obs = 3
obs_ls = []
input_f.write('map:\n')
input_f.write('    dimensions: [' + str(map_size_x) + ', ' + str(map_size_y) + ']\n')
input_f.write('    obstacles:' + '\n')
print('map:')
print('    dimensions: [', map_size_x, ', ', map_size_y, ']')
print('    obstacles:')
for i in range(num_static_obs):
    while True:
        obs_x = random.choice(range(map_size_x))
        obs_y = random.choice(range(map_size_y))
        if [obs_x, obs_y] not in start_ls and [obs_x, obs_y] not in end_ls and [obs_x, obs_y] not in obs_ls:
            obs_ls.append([obs_x, obs_y])
            break
        else:
            continue
    input_f.write('    - !!python/tuple ' + str([obs_x, obs_y]) + '\n')
    print('    - !!python/tuple ', [obs_x, obs_y])


# print(obs_ls)

map:
    dimensions: [ 32 ,  32 ]
    obstacles:
    - !!python/tuple  [12, 17]
    - !!python/tuple  [19, 16]
    - !!python/tuple  [22, 31]
    - !!python/tuple  [13, 24]
    - !!python/tuple  [13, 18]
    - !!python/tuple  [6, 1]
    - !!python/tuple  [7, 0]
    - !!python/tuple  [18, 8]
    - !!python/tuple  [19, 27]
    - !!python/tuple  [22, 20]
    - !!python/tuple  [0, 7]
    - !!python/tuple  [28, 28]
    - !!python/tuple  [22, 19]
    - !!python/tuple  [31, 7]
    - !!python/tuple  [24, 24]
    - !!python/tuple  [13, 0]
    - !!python/tuple  [17, 12]
    - !!python/tuple  [29, 26]
    - !!python/tuple  [19, 10]
    - !!python/tuple  [28, 12]
    - !!python/tuple  [23, 0]
    - !!python/tuple  [4, 31]
    - !!python/tuple  [15, 18]
    - !!python/tuple  [1, 26]
    - !!python/tuple  [9, 25]
    - !!python/tuple  [17, 11]
    - !!python/tuple  [4, 0]
    - !!python/tuple  [22, 16]
    - !!python/tuple  [26, 19]
    - !!python/tuple  [9, 29]
    - !!python/tuple  [16, 31]
    - 

In [86]:
input_f.write('obstacles:\n')
for i in range(num_dynamic_obs):
    input_f.write('-   start: [0, 0]\n')
    input_f.write('    goal: [32, 32]\n')
    input_f.write('    name: obstacle' + str(i) + '\n')
        
input_f.close()

dy_f = open(dy_file, 'a')

random.seed(seed)

dy_f.write('schedule:\n')
print('schedule:')
for dyn_obs in range(num_dynamic_obs):
    dy_f.write('  obstacle' + str(dyn_obs) + ':\n')
    print('  obstacle' + str(dyn_obs) + ':')
    while True:
        step_x = random.choice(range(map_size_x))
        step_y = random.choice(range(map_size_y))
        step = [step_x, step_y]
        
        is_bigger = True
        for start_pt in start_ls:
            manhadon_dist = abs(start_pt[0] - step_x) + abs(start_pt[1] - step_y)
            if manhadon_dist < manhadon_dist_min:
                is_bigger = False
                break
                
        if is_bigger:
            if step not in start_ls and step not in end_ls and step not in obs_ls:
                dy_f.write('  - t: ' + str(0) + '\n')
                dy_f.write('    x: ' + str(step_x) + '\n')
                dy_f.write('    y: ' + str(step_y) + '\n')
                print('  - t: ', 0)
                print('    x: ', step_x)
                print('    y: ', step_y)
                break
    step_num = random.choice(range(step_num_from, step_num_to))
    for step_ in range(step_num):
        while True:
            direction = random.choice(range(5)) # 0 => up | 1 => down | 2 => left | 3 => right | 4 => still 
            if direction == 0:
                step_x_ = step_x
                step_y_ = step_y + 1 
            elif direction == 1:
                step_x_ = step_x
                step_y_ = step_y - 1 
            elif direction == 2:
                step_y_ = step_y
                step_x_ = step_x - 1 
            elif direction == 3:
                step_y_ = step_y
                step_x_ = step_x + 1 
            else:
                step_y_ = step_y
                step_x_ = step_x

            step = [step_x_, step_y_]

            if step not in end_ls and step not in obs_ls and (0 <= step_x_ < map_size_x) and (0 <= step_y_ < map_size_y):
                dy_f.write('  - t: ' + str((step_ + 1)) + '\n')
                dy_f.write('    x: ' + str(step_x) + '\n')
                dy_f.write('    y: ' + str(step_y) + '\n')
                print('  - t: ', (step_ + 1))
                print('    x: ', step_x_)
                print('    y: ', step_y_)
                step_x = step_x_
                step_y = step_y_
                break
dy_f.close()

schedule:
  obstacle0:
  - t:  0
    x:  12
    y:  31


In [48]:
with open('inputs/input.yaml', 'r') as file:
    try:
        input_file = yaml.load(file, Loader=yaml.FullLoader)
    except yaml.YAMLError as exc:
        print(exc)
        
start_ls = []
end_ls = []
        
for agent in input_file['agents']:
    start_ls.append(agent['start'])
    end_ls.append(agent['goal'])

In [8]:
input_file

{'agents': [{'start': [30, 17], 'goal': [22, 9], 'name': 'agent0'},
  {'start': [24, 0], 'goal': [17, 29], 'name': 'agent1'},
  {'start': [14, 0], 'goal': [9, 28], 'name': 'agent2'},
  {'start': [23, 10], 'goal': [21, 13], 'name': 'agent3'},
  {'start': [3, 12], 'goal': [25, 5], 'name': 'agent4'},
  {'start': [1, 3], 'goal': [14, 5], 'name': 'agent5'},
  {'start': [27, 28], 'goal': [7, 27], 'name': 'agent6'},
  {'start': [8, 20], 'goal': [10, 5], 'name': 'agent7'},
  {'start': [30, 30], 'goal': [5, 12], 'name': 'agent8'},
  {'start': [16, 22], 'goal': [23, 24], 'name': 'agent9'}],
 'map': {'dimensions': [32, 32],
  'obstacles': [(0, 7),
   (0, 13),
   (0, 22),
   (0, 27),
   (0, 28),
   (0, 29),
   (1, 5),
   (1, 8),
   (1, 11),
   (1, 17),
   (1, 21),
   (1, 23),
   (1, 24),
   (2, 0),
   (2, 1),
   (2, 3),
   (2, 10),
   (2, 14),
   (2, 25),
   (3, 1),
   (3, 5),
   (3, 6),
   (3, 8),
   (3, 9),
   (3, 10),
   (3, 24),
   (4, 6),
   (4, 13),
   (4, 16),
   (4, 18),
   (4, 19),
   (4,

In [48]:
output = {'schedule': {'agent0': [{'t': '0', 'x': 24, 'y': 26}, {'t': '1', 'x': 23, 'y': 26}, {'t': '2', 'x': 23, 'y': 25}, {'t': '3', 'x': 22, 'y': 25}, {'t': '4', 'x': 21, 'y': 25}, {'t': '5', 'x': 20, 'y': 25}, {'t': '6', 'x': 19, 'y': 25}, {'t': '7', 'x': 18, 'y': 25}, {'t': '8', 'x': 17, 'y': 25}, {'t': '9', 'x': 17, 'y': 24}, {'t': '10', 'x': 16, 'y': 24}, {'t': '11', 'x': 15, 'y': 24}, {'t': '12', 'x': 14, 'y': 24}, {'t': '13', 'x': 13, 'y': 24}, {'t': '14', 'x': 12, 'y': 24}, {'t': '15', 'x': 11, 'y': 24}, {'t': '16', 'x': 11, 'y': 23}, {'t': 17, 'x': 2, 'y': 16}, {'t': 18, 'x': 2, 'y': 16}], 'agent1': [{'t': '0', 'x': 31, 'y': 25}, {'t': '1', 'x': 31, 'y': 26}, {'t': '2', 'x': 31, 'y': 27}, {'t': '3', 'x': 30, 'y': 27}, {'t': '4', 'x': 30, 'y': 28}, {'t': '5', 'x': 29, 'y': 28}, {'t': '6', 'x': 28, 'y': 28}, {'t': '7', 'x': 28, 'y': 29}, {'t': '8', 'x': 27, 'y': 29}, {'t': '9', 'x': 26, 'y': 29}, {'t': '10', 'x': 25, 'y': 29}, {'t': '11', 'x': 24, 'y': 29}, {'t': '12', 'x': 24, 'y': 30}, {'t': '13', 'x': 23, 'y': 30}, {'t': '14', 'x': 22, 'y': 30}, {'t': '15', 'x': 21, 'y': 30}, {'t': '16', 'x': 20, 'y': 30}, {'t': 17, 'x': 19, 'y': 30}, {'t': 18, 'x': 19, 'y': 30}], 'agent2': [{'t': '0', 'x': 22, 'y': 13}, {'t': '1', 'x': 22, 'y': 14}, {'t': '2', 'x': 21, 'y': 14}, {'t': '3', 'x': 20, 'y': 14}, {'t': '4', 'x': 20, 'y': 15}, {'t': '5', 'x': 19, 'y': 15}, {'t': '6', 'x': 19, 'y': 16}, {'t': '7', 'x': 18, 'y': 16}, {'t': '8', 'x': 17, 'y': 16}, {'t': '9', 'x': 17, 'y': 17}, {'t': '10', 'x': 16, 'y': 17}, {'t': '11', 'x': 15, 'y': 17}, {'t': '12', 'x': 14, 'y': 17}, {'t': '13', 'x': 13, 'y': 17}, {'t': '14', 'x': 13, 'y': 18}, {'t': '15', 'x': 12, 'y': 18}, {'t': '16', 'x': 11, 'y': 18}, {'t': 17, 'x': 8, 'y': 18}, {'t': 18, 'x': 8, 'y': 18}], 'agent3': [{'t': '0', 'x': 8, 'y': 6}, {'t': '1', 'x': 9, 'y': 6}, {'t': '2', 'x': 10, 'y': 6}, {'t': '3', 'x': 10, 'y': 7}, {'t': '4', 'x': 11, 'y': 7}, {'t': '5', 'x': 12, 'y': 7}, {'t': '6', 'x': 13, 'y': 7}, {'t': '7', 'x': 13, 'y': 8}, {'t': '8', 'x': 14, 'y': 8}, {'t': '9', 'x': 15, 'y': 8}, {'t': '10', 'x': 15, 'y': 9}, {'t': '11', 'x': 16, 'y': 9}, {'t': '12', 'x': 16, 'y': 9}, {'t': '13', 'x': 16, 'y': 9}, {'t': '14', 'x': 16, 'y': 9}, {'t': '15', 'x': 16, 'y': 9}, {'t': '16', 'x': 16, 'y': 9}, {'t': 17, 'x': 16, 'y': 9}, {'t': 18, 'x': 16, 'y': 9}], 'agent4': [{'t': '0', 'x': 19, 'y': 6}, {'t': '1', 'x': 18, 'y': 6}, {'t': '2', 'x': 18, 'y': 7}, {'t': '3', 'x': 18, 'y': 8}, {'t': '4', 'x': 17, 'y': 8}, {'t': '5', 'x': 16, 'y': 8}, {'t': '6', 'x': 16, 'y': 9}, {'t': '7', 'x': 16, 'y': 10}, {'t': '8', 'x': 15, 'y': 10}, {'t': '9', 'x': 14, 'y': 10}, {'t': '10', 'x': 14, 'y': 11}, {'t': '11', 'x': 13, 'y': 11}, {'t': '12', 'x': 14, 'y': 11}, {'t': '13', 'x': 13, 'y': 11}, {'t': '14', 'x': 14, 'y': 11}, {'t': '15', 'x': 13, 'y': 11}, {'t': '16', 'x': 13, 'y': 12}, {'t': 17, 'x': 5, 'y': 21}, {'t': 18, 'x': 4, 'y': 21}], 'agent5': [{'t': '0', 'x': 30, 'y': 6}, {'t': '1', 'x': 29, 'y': 6}, {'t': '2', 'x': 29, 'y': 7}, {'t': '3', 'x': 29, 'y': 8}, {'t': '4', 'x': 29, 'y': 9}, {'t': '5', 'x': 29, 'y': 10}, {'t': '6', 'x': 29, 'y': 11}, {'t': '7', 'x': 29, 'y': 12}, {'t': '8', 'x': 28, 'y': 12}, {'t': '9', 'x': 27, 'y': 12}, {'t': '10', 'x': 27, 'y': 13}, {'t': '11', 'x': 27, 'y': 14}, {'t': '12', 'x': 27, 'y': 15}, {'t': '13', 'x': 27, 'y': 16}, {'t': '14', 'x': 27, 'y': 17}, {'t': '15', 'x': 27, 'y': 18}, {'t': '16', 'x': 27, 'y': 19}, {'t': 17, 'x': 22, 'y': 27}, {'t': 18, 'x': 22, 'y': 27}], 'agent6': [{'t': '0', 'x': 20, 'y': 13}, {'t': '1', 'x': 20, 'y': 14}, {'t': '2', 'x': 20, 'y': 15}, {'t': '3', 'x': 21, 'y': 15}, {'t': '4', 'x': 21, 'y': 16}, {'t': '5', 'x': 22, 'y': 16}, {'t': '6', 'x': 23, 'y': 16}, {'t': '7', 'x': 23, 'y': 17}, {'t': '8', 'x': 23, 'y': 18}, {'t': '9', 'x': 24, 'y': 18}, {'t': '10', 'x': 24, 'y': 19}, {'t': '11', 'x': 24, 'y': 20}, {'t': '12', 'x': 24, 'y': 21}, {'t': '13', 'x': 25, 'y': 21}, {'t': '14', 'x': 26, 'y': 21}, {'t': '15', 'x': 27, 'y': 21}, {'t': '16', 'x': 28, 'y': 21}, {'t': 17, 'x': 30, 'y': 28}, {'t': 18, 'x': 30, 'y': 28}], 'agent7': [{'t': '0', 'x': 16, 'y': 3}, {'t': '1', 'x': 16, 'y': 4}, {'t': '2', 'x': 15, 'y': 4}, {'t': '3', 'x': 14, 'y': 4}, {'t': '4', 'x': 13, 'y': 4}, {'t': '5', 'x': 12, 'y': 4}, {'t': '6', 'x': 11, 'y': 4}, {'t': '7', 'x': 10, 'y': 4}, {'t': '8', 'x': 9, 'y': 4}, {'t': '9', 'x': 8, 'y': 4}, {'t': '10', 'x': 7, 'y': 4}, {'t': '11', 'x': 6, 'y': 4}, {'t': '12', 'x': 5, 'y': 4}, {'t': '13', 'x': 4, 'y': 4}, {'t': '14', 'x': 3, 'y': 4}, {'t': '15', 'x': 2, 'y': 4}, {'t': '16', 'x': 1, 'y': 4}, {'t': 17, 'x': 0, 'y': 5}, {'t': 18, 'x': 0, 'y': 5}], 'agent8': [{'t': '0', 'x': 25, 'y': 0}, {'t': '1', 'x': 25, 'y': 1}, {'t': '2', 'x': 25, 'y': 2}, {'t': '3', 'x': 25, 'y': 3}, {'t': '4', 'x': 25, 'y': 4}, {'t': '5', 'x': 25, 'y': 5}, {'t': '6', 'x': 25, 'y': 6}, {'t': '7', 'x': 25, 'y': 7}, {'t': '8', 'x': 25, 'y': 8}, {'t': '9', 'x': 25, 'y': 9}, {'t': '10', 'x': 26, 'y': 9}, {'t': '11', 'x': 26, 'y': 10}, {'t': '12', 'x': 26, 'y': 11}, {'t': '13', 'x': 26, 'y': 12}, {'t': '14', 'x': 26, 'y': 13}, {'t': '15', 'x': 26, 'y': 14}, {'t': '16', 'x': 26, 'y': 15}, {'t': 17, 'x': 31, 'y': 21}, {'t': 18, 'x': 31, 'y': 21}], 'agent9': [{'t': '0', 'x': 15, 'y': 20}, {'t': '1', 'x': 15, 'y': 19}, {'t': '2', 'x': 15, 'y': 18}, {'t': '3', 'x': 14, 'y': 18}, {'t': '4', 'x': 13, 'y': 18}, {'t': '5', 'x': 12, 'y': 18}, {'t': '6', 'x': 11, 'y': 18}, {'t': '7', 'x': 12, 'y': 18}, {'t': '8', 'x': 11, 'y': 18}, {'t': '9', 'x': 12, 'y': 18}, {'t': '10', 'x': 11, 'y': 18}, {'t': '11', 'x': 11, 'y': 17}, {'t': '12', 'x': 11, 'y': 16}, {'t': '13', 'x': 10, 'y': 16}, {'t': '14', 'x': 9, 'y': 16}, {'t': '15', 'x': 9, 'y': 15}, {'t': '16', 'x': 9, 'y': 14}, {'t': 17, 'x': 4, 'y': 12}, {'t': 18, 'x': 4, 'y': 12}], 'agent10': [{'t': '0', 'x': 14, 'y': 15}, {'t': '1', 'x': 14, 'y': 16}, {'t': '2', 'x': 13, 'y': 16}, {'t': '3', 'x': 13, 'y': 17}, {'t': '4', 'x': 12, 'y': 17}, {'t': '5', 'x': 11, 'y': 17}, {'t': '6', 'x': 10, 'y': 17}, {'t': '7', 'x': 9, 'y': 17}, {'t': '8', 'x': 9, 'y': 18}, {'t': '9', 'x': 9, 'y': 19}, {'t': '10', 'x': 9, 'y': 20}, {'t': '11', 'x': 9, 'y': 21}, {'t': '12', 'x': 9, 'y': 22}, {'t': '13', 'x': 9, 'y': 23}, {'t': '14', 'x': 9, 'y': 24}, {'t': '15', 'x': 9, 'y': 25}, {'t': '16', 'x': 9, 'y': 26}, {'t': 17, 'x': 9, 'y': 28}, {'t': 18, 'x': 9, 'y': 28}], 'agent11': [{'t': '0', 'x': 5, 'y': 5}, {'t': '1', 'x': 6, 'y': 5}, {'t': '2', 'x': 7, 'y': 5}, {'t': '3', 'x': 8, 'y': 5}, {'t': '4', 'x': 8, 'y': 6}, {'t': '5', 'x': 9, 'y': 6}, {'t': '6', 'x': 10, 'y': 6}, {'t': '7', 'x': 11, 'y': 6}, {'t': '8', 'x': 11, 'y': 7}, {'t': '9', 'x': 11, 'y': 8}, {'t': '10', 'x': 11, 'y': 9}, {'t': '11', 'x': 12, 'y': 9}, {'t': '12', 'x': 12, 'y': 10}, {'t': '13', 'x': 11, 'y': 10}, {'t': '14', 'x': 12, 'y': 10}, {'t': '15', 'x': 11, 'y': 10}, {'t': '16', 'x': 11, 'y': 11}, {'t': 17, 'x': 15, 'y': 24}, {'t': 18, 'x': 18, 'y': 27}], 'agent12': [{'t': '0', 'x': 6, 'y': 19}, {'t': '1', 'x': 6, 'y': 18}, {'t': '2', 'x': 7, 'y': 18}, {'t': '3', 'x': 8, 'y': 18}, {'t': '4', 'x': 9, 'y': 18}, {'t': '5', 'x': 10, 'y': 18}, {'t': '6', 'x': 9, 'y': 18}, {'t': '7', 'x': 10, 'y': 18}, {'t': '8', 'x': 9, 'y': 18}, {'t': '9', 'x': 10, 'y': 18}, {'t': '10', 'x': 9, 'y': 18}, {'t': '11', 'x': 9, 'y': 17}, {'t': '12', 'x': 10, 'y': 17}, {'t': '13', 'x': 11, 'y': 17}, {'t': '14', 'x': 12, 'y': 17}, {'t': '15', 'x': 13, 'y': 17}, {'t': '16', 'x': 13, 'y': 16}, {'t': 17, 'x': 17, 'y': 9}, {'t': 18, 'x': 18, 'y': 7}], 'agent13': [{'t': '0', 'x': 21, 'y': 13}, {'t': '1', 'x': 21, 'y': 14}, {'t': '2', 'x': 21, 'y': 15}, {'t': '3', 'x': 21, 'y': 16}, {'t': '4', 'x': 21, 'y': 17}, {'t': '5', 'x': 21, 'y': 18}, {'t': '6', 'x': 21, 'y': 19}, {'t': '7', 'x': 21, 'y': 20}, {'t': '8', 'x': 21, 'y': 21}, {'t': '9', 'x': 21, 'y': 22}, {'t': '10', 'x': 21, 'y': 23}, {'t': '11', 'x': 21, 'y': 24}, {'t': '12', 'x': 21, 'y': 25}, {'t': '13', 'x': 20, 'y': 25}, {'t': '14', 'x': 20, 'y': 26}, {'t': '15', 'x': 20, 'y': 27}, {'t': '16', 'x': 19, 'y': 27}, {'t': 17, 'x': 18, 'y': 28}, {'t': 18, 'x': 18, 'y': 28}], 'agent14': [{'t': '0', 'x': 5, 'y': 24}, {'t': '1', 'x': 5, 'y': 23}, {'t': '2', 'x': 6, 'y': 23}, {'t': '3', 'x': 7, 'y': 23}, {'t': '4', 'x': 8, 'y': 23}, {'t': '5', 'x': 9, 'y': 23}, {'t': '6', 'x': 10, 'y': 23}, {'t': '7', 'x': 11, 'y': 23}, {'t': '8', 'x': 12, 'y': 23}, {'t': '9', 'x': 12, 'y': 22}, {'t': '10', 'x': 13, 'y': 22}, {'t': '11', 'x': 13, 'y': 21}, {'t': '12', 'x': 14, 'y': 21}, {'t': '13', 'x': 14, 'y': 20}, {'t': '14', 'x': 14, 'y': 19}, {'t': '15', 'x': 14, 'y': 18}, {'t': '16', 'x': 14, 'y': 17}, {'t': 17, 'x': 20, 'y': 15}, {'t': 18, 'x': 20, 'y': 15}], 'agent15': [{'t': '0', 'x': 18, 'y': 11}, {'t': '1', 'x': 17, 'y': 11}, {'t': '2', 'x': 16, 'y': 11}, {'t': '3', 'x': 15, 'y': 11}, {'t': '4', 'x': 14, 'y': 11}, {'t': '5', 'x': 13, 'y': 11}, {'t': '6', 'x': 12, 'y': 11}, {'t': '7', 'x': 12, 'y': 11}, {'t': '8', 'x': 12, 'y': 11}, {'t': '9', 'x': 12, 'y': 11}, {'t': '10', 'x': 12, 'y': 11}, {'t': '11', 'x': 12, 'y': 11}, {'t': '12', 'x': 12, 'y': 11}, {'t': '13', 'x': 12, 'y': 11}, {'t': '14', 'x': 12, 'y': 11}, {'t': '15', 'x': 12, 'y': 11}, {'t': '16', 'x': 12, 'y': 11}, {'t': 17, 'x': 12, 'y': 11}, {'t': 18, 'x': 12, 'y': 11}], 'agent16': [{'t': '0', 'x': 2, 'y': 16}, {'t': '1', 'x': 2, 'y': 15}, {'t': '2', 'x': 3, 'y': 15}, {'t': '3', 'x': 4, 'y': 15}, {'t': '4', 'x': 5, 'y': 15}, {'t': '5', 'x': 6, 'y': 15}, {'t': '6', 'x': 7, 'y': 15}, {'t': '7', 'x': 8, 'y': 15}, {'t': '8', 'x': 8, 'y': 14}, {'t': '9', 'x': 8, 'y': 13}, {'t': '10', 'x': 9, 'y': 13}, {'t': '11', 'x': 10, 'y': 13}, {'t': '12', 'x': 11, 'y': 13}, {'t': '13', 'x': 12, 'y': 13}, {'t': '14', 'x': 13, 'y': 13}, {'t': '15', 'x': 14, 'y': 13}, {'t': '16', 'x': 15, 'y': 13}, {'t': 17, 'x': 26, 'y': 7}, {'t': 18, 'x': 30, 'y': 4}], 'agent17': [{'t': '0', 'x': 5, 'y': 8}, {'t': '1', 'x': 5, 'y': 7}, {'t': '2', 'x': 6, 'y': 7}, {'t': '3', 'x': 6, 'y': 6}, {'t': '4', 'x': 6, 'y': 5}, {'t': '5', 'x': 6, 'y': 4}, {'t': '6', 'x': 6, 'y': 3}, {'t': '7', 'x': 6, 'y': 2}, {'t': '8', 'x': 7, 'y': 2}, {'t': '9', 'x': 8, 'y': 2}, {'t': '10', 'x': 9, 'y': 2}, {'t': '11', 'x': 9, 'y': 2}, {'t': '12', 'x': 9, 'y': 2}, {'t': '13', 'x': 9, 'y': 2}, {'t': '14', 'x': 9, 'y': 2}, {'t': '15', 'x': 9, 'y': 2}, {'t': '16', 'x': 9, 'y': 2}, {'t': 17, 'x': 9, 'y': 2}, {'t': 18, 'x': 9, 'y': 2}], 'agent18': [{'t': '0', 'x': 5, 'y': 25}, {'t': '1', 'x': 5, 'y': 24}, {'t': '2', 'x': 5, 'y': 23}, {'t': '3', 'x': 6, 'y': 23}, {'t': '4', 'x': 7, 'y': 23}, {'t': '5', 'x': 8, 'y': 23}, {'t': '6', 'x': 9, 'y': 23}, {'t': '7', 'x': 10, 'y': 23}, {'t': '8', 'x': 10, 'y': 22}, {'t': '9', 'x': 11, 'y': 22}, {'t': '10', 'x': 11, 'y': 21}, {'t': '11', 'x': 11, 'y': 20}, {'t': '12', 'x': 11, 'y': 19}, {'t': '13', 'x': 11, 'y': 18}, {'t': '14', 'x': 11, 'y': 17}, {'t': '15', 'x': 12, 'y': 17}, {'t': '16', 'x': 12, 'y': 16}, {'t': 17, 'x': 17, 'y': 15}, {'t': 18, 'x': 17, 'y': 15}], 'agent19': [{'t': '0', 'x': 13, 'y': 26}, {'t': '1', 'x': 13, 'y': 27}, {'t': '2', 'x': 14, 'y': 27}, {'t': '3', 'x': 15, 'y': 27}, {'t': '4', 'x': 16, 'y': 27}, {'t': '5', 'x': 16, 'y': 28}, {'t': '6', 'x': 17, 'y': 28}, {'t': '7', 'x': 17, 'y': 28}, {'t': '8', 'x': 17, 'y': 28}, {'t': '9', 'x': 17, 'y': 28}, {'t': '10', 'x': 17, 'y': 28}, {'t': '11', 'x': 17, 'y': 28}, {'t': '12', 'x': 17, 'y': 28}, {'t': '13', 'x': 17, 'y': 28}, {'t': '14', 'x': 17, 'y': 28}, {'t': '15', 'x': 17, 'y': 28}, {'t': '16', 'x': 17, 'y': 28}, {'t': 17, 'x': 17, 'y': 28}, {'t': 18, 'x': 17, 'y': 28}]}}

with open('tmp.yaml', 'w') as output_yaml:
    yaml.safe_dump(output, output_yaml) 



In [53]:
import numpy as np
np.array([ 5, 23])[0]

5

In [10]:
import logging
logger = logging.getLogger('ABCBS_logger')
logger.setLevel('INFO')
logger.info('aaa')

INFO:ABCBS_logger:aaa
